In [1]:
''' Traveling Salesman Problem using Simulated Annealing '''
%matplotlib inline
import plotly.offline as py
import matplotlib.pyplot as plt
import random
import time

In [2]:
class City:
    '''
    City object
    1. x : x coordinate of the city
    2. y : y coordinate of the city
    3. front : connection in the front of the city
    4. back : connection in the back of the city
    '''
    front = None
    back = None

    def __init__(self, x, y):
        '''Initialize the coordinate of the city'''
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"City at Coordinate({self.x}, {self.y})"

    def add_connection(self, front, back):
        '''Add Connection to the city'''
        self.front = front
        self.back = back

    def swap_connection(self):
        '''Swap the Connection to the city'''
        self.front, self.back = self.back, self.front


class Tour:
    '''
    Tour / Route object
    1. xList : list of x coordinate of the tour
    2. yList : list of y coordinate of the tour
    3. tour : list of cities in order
    4. distance : total distance traveled
    '''

    def __init__(self, xList, yList, tour, distance):
        '''Initialize the information about the tour'''
        self.xList = xList
        self.yList = yList
        self.tour = tour
        self.distance = distance

    def set_attributes(self, other):
        '''Set the object attributes to match other object attributes'''
        self.xList = other.xList
        self.yList = other.yList
        self.tour = other.tour
        self.distance = other.distance

In [3]:
''' 2-Opt Edges Swap Animation '''
import random
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

random.seed(1437)

# Helping Function
def GenerateCity(cities):
    '''
    Make dictionary of City objects
    1. cities : coordinate data of cities
    --> returns dictionary of City objects
    '''
    result = {}
    for i, data in enumerate(cities):
        result[i] = City(data[0], data[1])
    return result

def Euclidean_dist(a, b):
    '''
    Calculate euclidean distance of 2 given points
    1. a : first point, a city object
    2. b : second point, a city object
    --> returns eauclidean distance
    '''
    return ((a.x - b.x)**2 + (a.y - b.y)**2)**0.5

# paramerters
init_notebook_mode(connected=True)
radiusX = 1000
radiusY = 1000
city_number = 15
frames = []

# Generate Random City Coordinate
cities = [(random.randrange(radiusX), random.randrange(radiusY)) for x in range(city_number)]
print(f"Cities : {cities}")

# Preparing Tour
start_point = random.randrange(city_number)

# Make JSON format of the cities
city_data = GenerateCity(cities)

# Generate Random Initial Tour
tour = random.sample(range(city_number), city_number)

# Updating cities data to include connection to each city and data to add as city markers
starting_x = []
starting_y = []
distance = 0
for i, num in enumerate(tour):
    starting_x.append(city_data[num].x)
    starting_y.append(city_data[num].y)
    city_data[num].add_connection(tour[(i - 1) % len(tour)], tour[(i + 1) % len(tour)])
    distance += Euclidean_dist(city_data[num], city_data[city_data[num].back])

starting_x.append(starting_x[0])
starting_y.append(starting_y[0])
distance += Euclidean_dist(city_data[tour[-1]], city_data[tour[0]])
city_markers = go.Scatter(x=starting_x, y=starting_y, mode='markers', name='City')

# Applying 2-Opt to get better routes
counter = 0  # number of attempted edges swap
success = 0  # number of successful edges swap
unchecked = []  # all possible combination of edges swap
for a in range(city_number):
    for b in range(city_number):
        if a != b:
            unchecked.append((a,b))
checked = []
steps = []

# everytime a successful edges swap occur, unchecked edges swap got reset
# only stop while all possible edges swap not resulting in better distances
while True:
    if not unchecked:
        # stop iteration if unchecked is empty
        print(f"Number of 2-Opt Operation : {counter}")
        print(f"Number of Successful 2-Opt Operation : {success}")
        break

    # picking 2 edges randomly
    edge_pair = random.choice(unchecked)
    i = edge_pair[0]
    k = edge_pair[1]

    # pick the point that will be swapped
    i2 = city_data[edge_pair[0]].back
    k2 = city_data[edge_pair[1]].front
    
    # check if 2 edges only have 3 point or not (the edges are not separate)
    if k2 == i or i2 == k:
        unchecked.remove(edge_pair)
        checked.append(edge_pair)
        continue
    
    # Calculate Old and New tour distance difference
    original = Euclidean_dist(city_data[i], city_data[i2])
    original += Euclidean_dist(city_data[k], city_data[k2])
    new_swap = Euclidean_dist(city_data[i], city_data[k2])
    new_swap += Euclidean_dist(city_data[k], city_data[i2])
    
    if new_swap < original:
        # Changing Connection from the Original
        city_data[i].back = k2
        city_data[k].front = i2
        city_data[i2].front = k
        city_data[k2].back = i
        # Reverse the Loop/Flow of the i2 - k2 connection
        curr = k2
        while True:
            city_data[curr].swap_connection()
            if curr == i2:
                break
            else:
                curr = city_data[curr].back
        # reset num_fail or restore edges_set and clear checked, and update distance, and update steps
        success += 1
        distance -= original
        distance += new_swap
        steps.append({
            'args': [
                        [success],
                        {
                            'frame': {'duration': 300, 'redraw': False},
                            'mode': 'immediate',
                            'transition': {'duration': 300}}
                    ],
             'label': success,
             'method': 'animate'
        })
        unchecked.extend(checked)
        checked.clear()
        # if success % 100 == 0:
        #     print("-->log:'{} Successful Edges Swapped'".format(success))
        
        # 1. get x and y data, turn it into list
        result = {
            'x': [],
            'y': []
        }
        current = 0
        while True:
            result['x'].append(city_data[current].x)
            result['y'].append(city_data[current].y)
            current = city_data[current].back
            
            if current == 0:
                # break if it goes back to starting city
                result['x'].append(city_data[current].x)
                result['y'].append(city_data[current].y)
                break
        
        # 2. update frames
        frames.append(go.Frame(
            data=[
                city_markers,
                go.Scatter(
                    x=result['x'],
                    y=result['y'],
                    mode='lines',
                    name='Route')
            ],
            name = str(success),
            layout = {'title': f"Successful Edges Swap No.{success} with {round(distance, 2)} Distance"}
        ))
    else:
        unchecked.remove(edge_pair)
        checked.append(edge_pair)
    
    # Logging for information during execution
    counter += 1
    if counter % 1000 == 0:
        print(f"At {counter} 2-Opt Operation...")

figure = {'data': [
                       {
                           'x': starting_x,
                           'y': starting_y,
                           'mode': 'markers'
                       },
                       {
                           'x': starting_x,
                           'y': starting_y,
                           'mode': 'lines',
                           'connectgaps': True
                       }
                  ],
          'layout': {
                        'xaxis': {'range': [0, radiusX], 'autorange': False},
                        'yaxis': {'range': [0, radiusY], 'autorange': False},
                        'title': 'Start',
                        'updatemenus': [
                            {
                                'buttons': [
                                    {
                                        'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                                 'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                                        'label': 'Play',
                                        'method': 'animate'
                                    },
                                    {
                                        'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                                        'transition': {'duration': 0}}],
                                        'label': 'Pause',
                                        'method': 'animate'
                                    }
                                ],
                                'direction': 'left',
                                'pad': {'r': 10, 't': 87},
                                'showactive': False,
                                'type': 'buttons',
                                'x': 0.1,
                                'xanchor': 'right',
                                'y': 0,
                                'yanchor': 'top'
                            }
                         ],
                         'sliders': [{
                            'active': 0,
                            'yanchor': 'top',
                            'xanchor': 'left',
                            'currentvalue': {
                                'font': {'size': 20},
                                'prefix': 'Success No.',
                                'visible': True,
                                'xanchor': 'right'
                            },
                            'transition': {'duration': 300, 'easing': 'cubic-in-out'},
                            'pad': {'b': 10, 't': 50},
                            'len': 0.9,
                            'x': 0.1,
                            'y': 0,
                            'steps': steps
                        }]
                    },
          'frames': frames
         }

iplot(figure)

Cities : [(221, 897), (798, 706), (607, 243), (808, 410), (355, 872), (718, 709), (345, 327), (472, 915), (559, 408), (559, 900), (508, 663), (41, 420), (89, 427), (429, 750), (706, 822)]
Number of 2-Opt Operation : 500
Number of Successful 2-Opt Operation : 18
